## Import Libraries and Start Spark Session

In [0]:
from pyspark.sql import SparkSession, DataFrame, functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col, year, month, dayofmonth, hour, rand, from_unixtime, date_format, floor, expr
from pyspark.sql.types import DoubleType
from functools import reduce
import math

# Initialize the Spark session
spark = SparkSession.builder \
    .appName("S3_JSON_Analysis") \
    .getOrCreate()

### Read data from S3 bucket and extract the year, month, day and hour

In [0]:
# Adjust the S3 path to match the root of your partitioned data
s3_path = "s3://computeractivity/"

# Load data with Spark, using pathGlobFilter for partition columns
df = (
    spark.read.option("basePath", s3_path)  # Sets base path for partition discovery
          .json(s3_path + "activity_type=*/year=*/month=*/day=*/hour=*")
)

# Now, Spark should recognize activity_type, year, month, day, and hour as columns
df.printSchema()  # Confirm schema includes the partition columns

# Extract and cast partition columns
df = df.withColumn("year", col("year").cast("int")) \
       .withColumn("month", col("month").cast("int")) \
       .withColumn("day", col("day").cast("int")) \
       .withColumn("hour", col("hour").cast("int")) \
       .withColumn("activity_type", col("activity_type"))

root
 |-- activity_type: string (nullable = true)
 |-- app_name: string (nullable = true)
 |-- button: string (nullable = true)
 |-- idle_duration: double (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- window_name: string (nullable = true)
 |-- word: string (nullable = true)
 |-- word_length: long (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)



In [0]:
app_change = df.filter("activity_type = 'app_change'")

app_counts = app_change.groupBy("app_name").count()
app_counts.show()

+-------------------+-----+
|           app_name|count|
+-------------------+-----+
|              Slack|  101|
|           Terminal|  160|
|AppleScript Utility|   64|
|      Google Chrome|  759|
|               main|    1|
|    Keychain Access|    1|
|    System Settings|   12|
|             Finder|    2|
|  Microsoft Outlook|    2|
|             Python|    2|
+-------------------+-----+



In [0]:
app_change.show()

+-------------+-------------+------+-------------+--------------------+--------------------+----+-----------+----+----+----+-----+---+----+----------+------------+
|activity_type|     app_name|button|idle_duration|           timestamp|         window_name|word|word_length|   x|   y|year|month|day|hour|   seconds|milliseconds|
+-------------+-------------+------+-------------+--------------------+--------------------+----+-----------+----+----+----+-----+---+----+----------+------------+
|   app_change|     Terminal|  NULL|         NULL|2024-10-26T17:28:...|willbutler — /Use...|NULL|       NULL|NULL|NULL|2024|   10| 26|  16|1729963732|         809|
|   app_change|     Terminal|  NULL|         NULL|2024-10-26T17:28:...|willbutler — will...|NULL|       NULL|NULL|NULL|2024|   10| 26|  16|1729963738|         136|
|   app_change|     Terminal|  NULL|         NULL|2024-10-26T17:29:...|willbutler — will...|NULL|       NULL|NULL|NULL|2024|   10| 26|  16|1729963741|         252|
|   app_change| 

In [0]:
words_typed = df.filter("activity_type = 'word_completed'")

words_by_date = words_typed.groupBy("year", "month", "day", "hour", "minute").agg(
    F.count("*").alias("words_typed"),
    F.avg("word_length").alias("avg_word_length")
)
words_by_date.show()

In [0]:
mouse_movements = df.filter("activity_type = 'mouse_movement'")

mouse_movements.show()

+--------------+--------+------+-------------+------------------+-----------+----+-----------+------------+------------+----+-----+---+----+----------+------------+--------------------+--------------------+----------+
| activity_type|app_name|button|idle_duration|         timestamp|window_name|word|word_length|           x|           y|year|month|day|hour|   seconds|milliseconds|      timestamp_test|     timestamp_mouse|      date|
+--------------+--------+------+-------------+------------------+-----------+----+-----------+------------+------------+----+-----+---+----+----------+------------+--------------------+--------------------+----------+
|mouse_movement|    NULL|  NULL|         NULL|1728821618.1731782|       NULL|NULL|       NULL| 453.3203125|783.41015625|2024|   10| 13|  12|1728821618|      173178|2024-10-13T12:13:...|2024-10-13T12:13:...|2024-10-13|
|mouse_movement|    NULL|  NULL|         NULL| 1728821618.326953|       NULL|NULL|       NULL| 453.3203125| 785.7890625|2024|   

In [0]:
mouse_movements.printSchema()

root
 |-- activity_type: string (nullable = true)
 |-- app_name: string (nullable = true)
 |-- button: string (nullable = true)
 |-- idle_duration: double (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- window_name: string (nullable = true)
 |-- word: string (nullable = true)
 |-- word_length: long (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- seconds: long (nullable = true)
 |-- milliseconds: integer (nullable = true)
 |-- timestamp_test: string (nullable = true)



In [0]:
mouse_clicks = df.filter("activity_type = 'mouse_click'")

mouse_clicks.show()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
dftime.show()

+--------------+--------+------+-------------+------------------+-----------+----+-----------+------------+------------+----+-----+---+----+----------+------------+--------------------+
| activity_type|app_name|button|idle_duration|         timestamp|window_name|word|word_length|           x|           y|year|month|day|hour|   seconds|milliseconds|      timestamp_test|
+--------------+--------+------+-------------+------------------+-----------+----+-----------+------------+------------+----+-----+---+----+----------+------------+--------------------+
|mouse_movement|    NULL|  NULL|         NULL|1728821618.1731782|       NULL|NULL|       NULL| 453.3203125|783.41015625|2024|   10| 13|  12|1728821618|         173|2024-10-13T12:13:...|
|mouse_movement|    NULL|  NULL|         NULL| 1728821618.326953|       NULL|NULL|       NULL| 453.3203125| 785.7890625|2024|   10| 13|  12|1728821618|         326|2024-10-13T12:13:...|
|mouse_movement|    NULL|  NULL|         NULL| 1728821618.352664|     

### Convert the Unix Timestamps for the mouse movements into Date Time with milliseconds

In [0]:
df = df.withColumn("seconds", floor(col("timestamp"))) \
       .withColumn("milliseconds", ((col("timestamp") - col("seconds")) * 1000000).cast("int"))

# Convert integer seconds to formatted datetime, add milliseconds part
df = df.withColumn("timestamp_mouse", 
                   F.when(F.col("activity_type").isin("mouse_movement", "mouse_click"),
                          expr("from_unixtime(seconds, 'yyyy-MM-dd') || 'T' || from_unixtime(seconds, 'HH:mm:ss') || '.' || lpad(milliseconds, 6, '0')")
                         ).otherwise(F.col("timestamp"))
                  )

df = df.withColumn("date", F.date_format("timestamp_mouse", "yyyy-MM-dd"))

In [0]:
print(df.limit(2).select("timestamp_mouse").collect()[1]["timestamp_mouse"])
print("Break")
print(app_change.limit(2).select('timestamp').collect()[1]['timestamp'])

2024-10-13T12:13:38.326952
Break
2024-10-26T17:28:58.136904


## Calculate the Distance and Angle of the mouse movements 

In [0]:
# Angle threshold for detecting direction changes
ANGLE_CHANGE_THRESHOLD = 70  # Degrees
MIN_DISTANCE_THRESHOLD = 50  # Minimum distance to consider as movement

# Calculate distance traveled for each row
def calculate_distance(x1, y1, x2, y2):
    if x1 is None or y1 is None or x2 is None or y2 is None:
        return 0.0
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def calculate_angle(x1, y1, x2, y2, x3, y3):
    if None in (x1, y1, x2, y2, x3, y3):
        return 0.0
    # Vectors (x1, y1) -> (x2, y2) and (x2, y2) -> (x3, y3)
    v1x, v1y = x2 - x1, y2 - y1
    v2x, v2y = x3 - x2, y3 - y2
    # Calculate magnitudes
    mag1 = math.sqrt(v1x**2 + v1y**2)
    mag2 = math.sqrt(v2x**2 + v2y**2)
    if mag1 < MIN_DISTANCE_THRESHOLD or mag2 < MIN_DISTANCE_THRESHOLD:
        return 0.0  # Ignore small movements
    # Calculate the cosine of the angle
    dot_product = v1x * v2x + v1y * v2y
    cos_theta = dot_product / (mag1 * mag2)
    # Clamp cos_theta to avoid domain errors
    cos_theta = max(-1, min(1, cos_theta))
    # Convert to angle in degrees
    angle = math.degrees(math.acos(cos_theta))
    return angle

# UDF to calculate distance
distance_udf = F.udf(calculate_distance, DoubleType())
# UDF for angle calculation
angle_udf = F.udf(calculate_angle, DoubleType())

# Lag to get previous coordinates
window_spec = Window.partitionBy("year", "month", "day", "hour", "minute").orderBy("timestamp")
df = df.withColumn("x_prev", F.lag("x", 1).over(window_spec))
df = df.withColumn("y_prev", F.lag("y", 1).over(window_spec))
df = df.withColumn("x_next", F.lead("x", 1).over(window_spec))
df = df.withColumn("y_next", F.lead("y", 1).over(window_spec))

df = df.withColumn("distance", distance_udf("x", "y", "x_prev", "y_prev"))
df = df.withColumn("angle", angle_udf("x_prev", "y_prev", "x", "y", "x_next", "y_next"))

# Calculate direction changes by comparing consecutive angles
df = df.withColumn("angle_prev", F.lag("angle").over(window_spec))

# Detect direction change based on angle exceeding threshold
df = df.withColumn("direction_change", F.when(F.col("angle") > ANGLE_CHANGE_THRESHOLD, 1).otherwise(0))

In [0]:
# Aggregation to calculate total distance and direction changes by minute
result = df.groupBy("year", "month", "day", "hour", "minute").agg(
    F.sum("distance").alias("total_distance"),
    F.sum("direction_change").alias("direction_changes"),
    F.sum(F.when(F.col("activity_type") == "word_completed",1).otherwise(0)).alias("total_words"),
    F.avg(F.when(F.col("activity_type") == "word_completed",F.col("word_length")).otherwise(None)).alias("avg_word_length"),
    F.sum(F.when(F.col("activity_type") == "mouse_click", 1).otherwise(0)).alias("clicks_count")
)

result = result.withColumn(
                "date",
                F.to_date(F.concat(F.col('year'), F.lit('-'), F.col('month'), F.lit('-'), F.col('day')), "yyyy-MM-dd")
)

result.show()

print(f'Number of rows in groupby = {result.count()}')

In [0]:
app_change = app_change.withColumn("timestamp_end", F.lead("timestamp").over(Window.orderBy('timestamp')))

mouse_movements = mouse_movements.alias("mouse_movements")
app_change = app_change.alias("app_change")

app_change_mouse_movements = app_change.join(mouse_movements,
                                             (F.col("mouse_movements.timestamp_mouse") >= F.col("app_change.timestamp")) &
                                             (F.col("mouse_movements.timestamp_mouse") < F.col("app_change.timestamp_end")),
                                             how="left" )


In [0]:
app_change_mouse_movements = app_change_mouse_movements.withColumn("timestamp_start_unix", F.unix_timestamp("app_change.timestamp", "yyyy-MM-dd'T'HH:mm:ss.SSSSSS"))
app_change_mouse_movements = app_change_mouse_movements.withColumn("timestamp_end_unix", F.unix_timestamp("app_change.timestamp_end", "yyyy-MM-dd'T'HH:mm:ss.SSSSSS"))

app_change_mouse_movements = app_change_mouse_movements.withColumn("time_diff_seconds", F.col('timestamp_end_unix') - F.col('timestamp_start_unix'))

# If you want the difference in a more readable format, like hours, minutes, etc.
app_change_mouse_movements = app_change_mouse_movements.withColumn("time_diff_minutes", F.col("time_diff_seconds") / 60)  # For minutes
app_change_mouse_movements = app_change_mouse_movements.withColumn("time_diff_hours", F.col("time_diff_seconds") / 3600)

app_change_mouse_movements = app_change_mouse_movements.withColumn("timestamp_range", F.concat(F.col("app_change.timestamp"), F.lit(" to "), F.col("app_change.timestamp_end")))

In [0]:
app_change_mouse_movements.select("app_change.app_name", "timestamp_range", "time_diff_seconds", "time_diff_minutes","mouse_movements.timestamp_mouse").show(truncate=False)

+-------------------+--------------------------------------------------------+-----------------+-----------------+--------------------------+
|app_name           |timestamp_range                                         |time_diff_seconds|time_diff_minutes|timestamp_mouse           |
+-------------------+--------------------------------------------------------+-----------------+-----------------+--------------------------+
|AppleScript Utility|2024-10-09T16:12:04.367325 to 2024-10-09T16:15:08.513721|184              |3.066666666666667|2024-10-09T16:14:14.262912|
|AppleScript Utility|2024-10-09T16:12:04.367325 to 2024-10-09T16:15:08.513721|184              |3.066666666666667|2024-10-09T16:14:14.479892|
|AppleScript Utility|2024-10-09T16:12:04.367325 to 2024-10-09T16:15:08.513721|184              |3.066666666666667|2024-10-09T16:14:14.508553|
|AppleScript Utility|2024-10-09T16:12:04.367325 to 2024-10-09T16:15:08.513721|184              |3.066666666666667|2024-10-09T16:14:14.537649|
|Apple

In [0]:
app_change_mouse_agg = app_change_mouse_movements.groupBy().agg()

In [0]:
print(result.count())

results_sample = result.sample(fraction=0.5).orderBy(rand()).limit(50)

display(results_sample)

In [0]:
result.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable("mouse_activity_summary")


In [0]:
app_change = app_change.withColumn(
    "end_datetime_msecs", 
    F.lead("datetime_msecs").over(Window.orderBy('timestamp'))
)

# Alias the DataFrames to avoid ambiguity
app_change_alias = app_change.alias("app_change")
mouse_movements_alias = mouse_movements.alias("mouse_movements")

# Perform the join with qualified column names
app_change_mouse_movements = app_change_alias.join(
    mouse_movements_alias,
    (mouse_movements_alias.datetime_msecs >= app_change_alias.datetime_msecs) &
    (mouse_movements_alias.datetime_msecs < app_change_alias.end_datetime_msecs),
    how="left"
)

display(app_change_mouse_movements)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1244502079990473>, line 11
      8 mouse_movements_alias = mouse_movements.alias("mouse_movements")
     10 # Perform the join with qualified column names
---> 11 app_change_mouse_movements = app_change_alias.join(
     12     mouse_movements_alias,
     13     (mouse_movements_alias.datetime_msecs >= app_change_alias.datetime_msecs) &
     14     (mouse_movements_alias.datetime_msecs < app_change_alias.end_datetime_msecs),
     15     how="left"
     16 )
     18 display(app_change_mouse_movements)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, sig